In [1]:
n_timesteps = 60 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 10 # n. barre nel futuro di cui si desidera prevedere il prezzo


print("Importa librerie", end="", flush=True)

#from keras.losses import Huber
#print(".", end="", flush=True)
import pandas as pd
print(".", end="", flush=True)
import yfinance as yf
print(".", end="", flush=True)
import numpy as np
print(".", end="", flush=True)
import tensorflow as tf
print(".", end="", flush=True)
import funzioni as fx
print(".", end="", flush=True)
from tensorflow.keras.models import load_model
print(".", end="", flush=True)
import os
print("=", end="\n", flush=True)

elenco_features = [
    "Close",
    "EMA_5", 
    "EMA_20", 
    "EMA_50",
    "Open",  
    "High",
    "Low",
    "Volume",
    "MACDh",
    "PSAR",
    "PSARaf",
    "SUPERT", 
    "TRIX",
    "ATR",
    "DM_OSC",
    "ADX"
]
elenco_targets = [
    "EMA_5",
    "EMA_20", 
    "EMA_50"    
]

col_features = {col: idx for idx, col in enumerate(elenco_features)}
col_targets = {col: idx for idx, col in enumerate(elenco_targets)}
n_features = len(col_features)
n_targets = len(col_targets)

from sklearn.preprocessing import PowerTransformer
#from sklearn.preprocessing import RobustScaler
from sklearn.base import clone
scaler = PowerTransformer()
#scaler = RobustScaler()

print("Download lista ticker")
lista_ticker = pd.read_parquet("Tickers_De_Giro.parquet")
lista_ticker = lista_ticker.loc[(lista_ticker["Categoria"] != "D"), :]

if os.path.exists('LSTM.keras'):
    print("Caricamento modello esistente")
    model = load_model("LSTM.keras")


Importa librerie......=
Download lista ticker
Caricamento modello esistente


In [2]:
screener = pd.DataFrame(columns=["Ticker", "Gain"])
col_analisi = "EMA_5"
for i_ticker in range (len(lista_ticker)):
    nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
    print(f"\033[48;5;42m{i_ticker+1} di {len(lista_ticker)}: Ticker {nome_simbolo}\033[0m")
    print("Download dati ticker")
    try:
        ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
        if ticker["Close"].iloc[-1] >= 1:
            print("Download dati ticker per previsione")
            dati_previsione = yf.download(nome_simbolo, start='2022-01-01', end='2023-12-31', progress=False)
            dati_previsione.index = dati_previsione.index.date
            dati_previsione = fx.crea_indicatori(dati_previsione)
            dati_previsione.iloc[:100] = dati_previsione.iloc[:100].dropna(axis=0)

            indice_prev, X_prev, Y_prev = fx.to_XY(dati_previsione, elenco_features, elenco_targets, n_timesteps, giorni_previsione, addestramento=False)

            X_prev = X_prev.reshape(-1, n_timesteps * n_features)
            Y_prev = Y_prev.reshape(-1, giorni_previsione * n_targets)
            X_scaler_prev = clone(scaler)
            Y_scaler_prev = clone(scaler)
            X_prev = X_scaler_prev.fit_transform(X_prev)
            Y_prev = Y_scaler_prev.fit_transform(Y_prev)
            X_prev = X_prev.reshape(-1, n_timesteps, n_features)
            Y_prev = Y_prev.reshape(-1, giorni_previsione, n_targets)

            pred = model.predict(X_prev)
            #pred_loss, pred_mae = model.evaluate(X_prev, Y_prev)

            X_prev = X_prev.reshape(-1, n_timesteps * n_features)
            X_prev = X_scaler_prev.inverse_transform(X_prev)
            X_prev = X_prev.reshape(-1, n_timesteps, n_features)

            pred = pred.reshape(-1, giorni_previsione * n_targets)
            pred = Y_scaler_prev.inverse_transform(pred)
            pred = pred.reshape(-1, giorni_previsione, n_targets)
            
            val_prev = pred[-1, giorni_previsione-1, col_targets[col_analisi]]
            ultimo_valore = X_prev[-1, n_timesteps-1, col_features[col_analisi]]
            gain = fx.pct_change(ultimo_valore, val_prev)
            screener = pd.concat([screener, pd.DataFrame({"Ticker": nome_simbolo, "Gain": gain})])
    except Exception as e:
        print(e)
        continue
print("fine")

1 di 2581: Ticker GES
Download dati ticker
Download dati ticker per previsione
12/12 [==============================] - 2s 21ms/step
If using all scalar values, you must pass an index
2 di 2581: Ticker FLWS
Download dati ticker
Download dati ticker per previsione
12/12 [==============================] - 0s 18ms/step
If using all scalar values, you must pass an index
3 di 2581: Ticker SRCE
Download dati ticker
Download dati ticker per previsione
12/12 [==============================] - 0s 24ms/step
If using all scalar values, you must pass an index
4 di 2581: Ticker QFIN
Download dati ticker


C:\Users\932197\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:3234: RuntimeWarning:

divide by zero encountered in power



Download dati ticker per previsione
12/12 [==============================] - 0s 21ms/step
If using all scalar values, you must pass an index
5 di 2581: Ticker MMM
Download dati ticker
Download dati ticker per previsione
12/12 [==============================] - 0s 19ms/step
If using all scalar values, you must pass an index
6 di 2581: Ticker ATEN
Download dati ticker
Download dati ticker per previsione
12/12 [==============================] - 0s 18ms/step
If using all scalar values, you must pass an index
7 di 2581: Ticker AAON
Download dati ticker


C:\Users\932197\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:3234: RuntimeWarning:

divide by zero encountered in power



Download dati ticker per previsione
12/12 [==============================] - 0s 20ms/step
If using all scalar values, you must pass an index
8 di 2581: Ticker AIR
Download dati ticker
Download dati ticker per previsione
12/12 [==============================] - 0s 21ms/step
If using all scalar values, you must pass an index
9 di 2581: Ticker AAN
Download dati ticker
Download dati ticker per previsione
12/12 [==============================] - 0s 19ms/step
If using all scalar values, you must pass an index
10 di 2581: Ticker ABT
Download dati ticker
Download dati ticker per previsione
12/12 [==============================] - 0s 20ms/step
If using all scalar values, you must pass an index
11 di 2581: Ticker ABBV
Download dati ticker
Download dati ticker per previsione
12/12 [==============================] - 0s 18ms/step
If using all scalar values, you must pass an index
12 di 2581: Ticker ABCL
Download dati ticker
Download dati ticker per previsione
12/12 [==============================] 

...

Download dati ticker per previsione
'Index' object has no attribute 'date'
2579 di 2581: Ticker 1VOW3
Download dati ticker



1 Failed download:
['1VOW3']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['WIIT']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
2580 di 2581: Ticker WIIT
Download dati ticker
single positional indexer is out-of-bounds
2581 di 2581: Ticker ZV
Download dati ticker



1 Failed download:
['ZV']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2022-01-01 -> 2023-12-31)')


Download dati ticker per previsione
'Index' object has no attribute 'date'
fine


In [3]:
screener.to_excel("screener.xlsx")

In [4]:
screener

Empty DataFrame
Columns: [Ticker, Gain]
Index: []